### Part - I: Importing Required Modules/ Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf 
import keras
print(tf.__version__)
print(keras.__version__)
print(pd.__version__)
print(np.__version__)

2.6.0
2.6.0
1.3.4
1.19.5


In [2]:
# custom imports 
from nn_globals import *
from dataset import muon_data_split
from nn_plotting import gaus, fit_gaus, corr_plot

### Part- II: Data Preprocessing

In [ ]:
# Import muon data
# 'x' is the array of input variables, 'y' is the q/pT
x_train_displ, x_test_displ, y_train_displ, y_test_displ, dxy_train_displ, dxy_test_displ= muon_data_split(filename=DATAFILEPATH, 
                                                                                                           reg_pt_scale=REG_PT_SCALE, 
                                                                                                           reg_dxy_scale=REG_DXY_SCALE, 
                                                                                                           test_size=TEST_SIZE,
                                                                                                           nvariables = NVARIABLES,
                                                                                                           nentries= NENTRIES,
                                                                                                           batch_size = 128)

### Part-III: Use the cells in this section for data exploration

In [ ]:
# cols = ["dphi_1","dphi_2","dphi_3","dphi_4","dphi_5","dphi_6",
#        "dtheta_1","dtheta_2","dtheta_3","dtheta_4","dtheta_5", "dtheta_6",
#        "bend_1","bend_2","bend_3","bend_4",
#        "track theta"]

# x = np.concatenate((x_train_displ,x_test_displ),axis=0)
# y = np.concatenate((y_train_displ,y_test_displ),axis=0)
# dxy = np.concatenate((dxy_train_displ,dxy_test_displ),axis=0)

# corr_plot(x,y,dxy,columns = cols)

### Part-IV: Build the custom model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, TerminateOnNaN, EarlyStopping

# project-specific imports
from nn_evaluate import huber_loss, k_fold_validation
from nn_training import lr_schedule
from nn_pruning_module_support import loading_trained_model
from nn_training_pruned_model import (generate_layer_masks, 
                                      create_sparse_model)
from nn_training import train_model

In [ ]:
baseline = loading_trained_model(filepath = "models/",
                                 model_filename = "model")
baseline.summary()

In [ ]:
# PRUNING CYCLE - I

l1_reg = 0.0
l2_reg = 0.0
eps = 1e-4
momentum = 0.9
lr = 1e-4
retrain_batch_size = 1000
sparsity = 0.10
retrain_epochs = 50
clipnorm = 10.0

adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_10 = create_sparse_model(model = baseline,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)
pruned_model_10, history_10 = train_model(model = pruned_model_10,
                                            x = x_train_displ,
                                            y = np.column_stack((y_train_displ,dxy_train_displ)),
                                            epochs = retrain_epochs,
                                            batch_size = retrain_batch_size,
                                            callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                            verbose = True,
                                            validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_10, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_10, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_10,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

In [ ]:
# PRUNING CYCLE - II

lr = 2.5e-4
clipnorm = 10.
eps = 1e-4
momentum = 0.9
retrain_epochs = 75
retrain_batch_size = 1000
l1_reg = 0.0
l2_reg = 0.0
sparsity = 0.20
adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_20 = create_sparse_model(model = pruned_model_10,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)

pruned_model_20, history_20 = train_model(model = pruned_model_20,
                                           x = x_train_displ,
                                           y = np.column_stack((y_train_displ,dxy_train_displ)),
                                           epochs = retrain_epochs,
                                           batch_size = retrain_batch_size,
                                           callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                           verbose = True,
                                           validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_20, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_20, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_20,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

In [ ]:
# PRUNING CYCLE - III

lr = 3e-4
clipnorm = 10.
eps = 1e-4
momentum = 0.9
retrain_epochs = 75
retrain_batch_size = 800
l1_reg = 0.0
l2_reg = 0.0

adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_30 = create_sparse_model(model = pruned_model_20,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)

pruned_model_30, history_30 = train_model(model = pruned_model_30,
                                           x = x_train_displ,
                                           y = np.column_stack((y_train_displ,dxy_train_displ)),
                                           epochs = retrain_epochs,
                                           batch_size = retrain_batch_size,
                                           callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                           verbose = True,
                                           validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_30, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_30, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_30,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

In [ ]:
# PRUNING CYCLE - IV

lr = 5e-4
clipnorm = 10.
eps = 1e-4
momentum = 0.8
retrain_epochs = 100
retrain_batch_size = 800
l1_reg = 0.0
l2_reg = 0.0

adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_40 = create_sparse_model(model = pruned_model_30,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)

pruned_model_40, history_40 = train_model(model = pruned_model_40,
                                           x = x_train_displ,
                                           y = np.column_stack((y_train_displ,dxy_train_displ)),
                                           epochs = retrain_epochs,
                                           batch_size = retrain_batch_size,
                                           callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                           verbose = True,
                                           validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_40, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_40, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_40,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

In [ ]:
# PRUNING CYCLE - V

lr = 7.5e-4
clipnorm = 10.
eps = 1e-4
momentum = 0.9
retrain_epochs = 125
retrain_batch_size = 500
l1_reg = 0.0
l2_reg = 0.0
sparsity = 0.50

adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_50 = create_sparse_model(model = pruned_model_40,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)

pruned_model_50, history_50 = train_model(model = pruned_model_50,
                                           x = x_train_displ,
                                           y = np.column_stack((y_train_displ,dxy_train_displ)),
                                           epochs = retrain_epochs,
                                           batch_size = retrain_batch_size,
                                           callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                           verbose = True,
                                           validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_50, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_50, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_50,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

In [ ]:
# PRUNING CYCLE - VI

lr = 3.5e-3
clipnorm = 10.
eps = 1e-4
momentum = 0.9
retrain_epochs = 200
retrain_batch_size = 1000
l1_reg = 0.0
l2_reg = 0.0
sparsity = 0.60

adam = Adam(lr=lr, clipnorm=clipnorm)
lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0,
                               mode='auto', baseline=None, restore_best_weights=False)

pruned_model_60 = create_sparse_model(model = pruned_model_50,
                                       input_dim = NVARIABLES,
                                       output_dim = 2,
                                       k_sparsity = sparsity,
                                       bn_epsilon = eps,
                                       bn_momentum = momentum,
                                       l1_reg = l1_reg,
                                       l2_reg = l1_reg,
                                       kernel_initializer="glorot_uniform",
                                       optimizer = adam)

pruned_model_60, history_60 = train_model(model = pruned_model_60,
                                           x = x_train_displ,
                                           y = np.column_stack((y_train_displ,dxy_train_displ)),
                                           epochs = retrain_epochs,
                                           batch_size = retrain_batch_size,
                                           callbacks=[lr_decay, 
                                                      early_stopping, 
                                                      terminate_on_nan],
                                           verbose = True,
                                           validation_split=0.1)

In [ ]:
k_fold_validation(model = pruned_model_60, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ,
                  folds =1,
                  metric_type = "RMSE")
k_fold_validation(model = pruned_model_60, 
                  x = x_test_displ,
                  y = y_test_displ,
                  dxy = dxy_test_displ, 
                  folds =1,
                  metric_type = "MAE")
__generate_delta_plots__(pruned_model_60,
                         x = x_test_displ,
                         y = y_test_displ,
                         dxy = dxy_test_displ,
                         plot_colors = "seagreen")

### Part- V: Saving the trained pruned model(s)

In [ ]:
from nn_pruning_module_support import saving_model

pruned_models = [pruned_model_10,pruned_model_20, pruned_model_30, pruned_model_40, pruned_model_50, pruned_model_60]

for i in range(len(pruned_models)):
    j = (i+1)*10
    saving_model(model = pruned_models[i],
                filepath = "./models",
                model_filename = "pruned_model_{}".format(j) )